# g) Ngonye Falls Power and Energy Periods
Produces 'simulations' of station operation over two periods, debt period and equity period. Each simulation has a different start date on the historic series.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
|    [Model Name]_pe_yearly.csv| Notebook: d_ewr_setup | Summary energy produciton by year of the historic series |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
| [Model Name]_pe_sims.csv | Detailed (yearly) output. |
| [Model Name]_pe_sims_summary.csv | Summary by start year  |



## Libraries and Setup

In [1]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [2]:
input_data='./input_data/'
output_data='./output_data/'

model_name='Base Case'


In [3]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="scenarios/" + out_prefix

debt_period=18
equity_period=25
discount_rate=1.0 + 0.15

In [4]:
yearly=pd.read_csv(output_data + out_prefix + "_pe_yearly.csv")

In [5]:
yearly

,WaterYear,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor,Prop4Units,Prop3Units,Prop2Units,Prop1Units,Prop0Units,Days0Units
0,1924,3456.928400,111.133124,0.872827,0.052937,25.408305,13.482746,974.580352,964.401972,189.887819,0.000000,827686.134030,0.524915,0.408219,0.035616,0.109589,0.419178,0.027397,10
1,1925,4504.008836,157.205464,0.566039,0.040630,25.355414,11.551136,976.511962,964.401972,193.747347,22.358325,767292.232182,0.486614,0.317808,0.076712,0.189041,0.416438,0.000000,0
2,1926,3305.305419,192.534369,0.496243,0.065652,25.437234,13.793090,974.270008,964.401972,190.872868,20.757348,857958.191273,0.544114,0.378082,0.060274,0.178082,0.383562,0.000000,0
3,1927,2256.674509,178.800013,0.473067,0.065583,25.390386,16.233306,971.829792,964.401972,189.332151,22.514464,826892.270682,0.524412,0.304110,0.106849,0.178082,0.413699,0.000000,0
4,1928,1587.035405,195.867340,0.502447,0.093256,25.387509,18.166094,969.897004,964.405963,192.801268,22.924151,747056.070205,0.473780,0.172603,0.145205,0.210959,0.471233,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2012,3617.058785,224.058948,0.505204,0.055725,25.076798,13.164406,974.898692,964.717318,191.420746,24.932292,843394.858484,0.534878,0.413699,0.076712,0.128767,0.380822,0.000000,0
89,2013,3578.198961,194.855402,0.497805,0.056469,25.437511,13.240804,974.822294,964.400513,191.638019,20.627822,926765.634148,0.587751,0.446575,0.131507,0.128767,0.293151,0.000000,0
90,2014,1460.437582,205.608866,0.542968,0.100587,25.105646,18.622820,969.485125,964.706250,192.445122,24.641825,885184.222151,0.561380,0.306849,0.117808,0.161644,0.413699,0.000000,0
91,2015,2973.367833,173.346774,0.501598,0.067263,25.390627,14.505749,973.557349,964.400513,191.219733,21.113961,787616.850388,0.499503,0.326027,0.068493,0.169863,0.438356,0.000000,0


In [24]:
sims=pd.DataFrame({'StartYear':np.arange(yearly['WaterYear'].min(),yearly['WaterYear'].max()-equity_period+2,1)})
sims['DebtEnd']=sims['StartYear']+debt_period-1
sims['EquityEnd']=sims['StartYear']+equity_period-1
sims['SimName']=sims['StartYear']-5
sims['EquivalentPeriod']=2022-sims['StartYear']
sims=sims.set_index('StartYear')
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod
StartYear,,,,
1924,1941,1948,1919,98
1925,1942,1949,1920,97
1926,1943,1950,1921,96
1927,1944,1951,1922,95
1928,1945,1952,1923,94
...,...,...,...,...
1988,2005,2012,1983,34
1989,2006,2013,1984,33
1990,2007,2014,1985,32


In [25]:
sims.loc[1967]

DebtEnd             1984
EquityEnd           1991
SimName             1962
EquivalentPeriod      55
Name: 1967, dtype: int64

In [26]:


o=pd.DataFrame()
for row in sims.itertuples(): 
    f=pd.DataFrame(yearly.loc[(yearly['WaterYear']>=row.Index) & (yearly['WaterYear']<=row.EquityEnd)].loc[:,['Energy']])
    f['Energy']=f['Energy']/1000
    f=f.reset_index()
    f['YearNo']=f.index+1
    f['StartYear']=row.Index
    f['WaterYear']=f['StartYear']+f.index
    f=f.set_index('StartYear').drop('index',axis=1)
    #display(f)
    
    o=o.append(f)

sims=sims.join(o)
#sims=sims.reset_index()
#sims=sims.set_index(['StartYear','YearNo'])
sims
#o

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear
StartYear,,,,,,,
1924,1941,1948,1919,98,827.686134,1,1924
1924,1941,1948,1919,98,767.292232,2,1925
1924,1941,1948,1919,98,857.958191,3,1926
1924,1941,1948,1919,98,826.892271,4,1927
1924,1941,1948,1919,98,747.056070,5,1928
...,...,...,...,...,...,...,...
1992,2009,2016,1987,30,843.394858,21,2012
1992,2009,2016,1987,30,926.765634,22,2013
1992,2009,2016,1987,30,885.184222,23,2014


In [27]:
flow_yearly=pd.read_csv(output_data + "ngonye_flow_yearly.csv").set_index('WaterYear')
sims['Flow_mean']=sims.join(flow_yearly,on='WaterYear',lsuffix='l_')['Flow_mean']
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear,Flow_mean
StartYear,,,,,,,,
1924,1941,1948,1919,98,827.686134,1,1924,991.317021
1924,1941,1948,1919,98,767.292232,2,1925,1111.041962
1924,1941,1948,1919,98,857.958191,3,1926,990.272631
1924,1941,1948,1919,98,826.892271,4,1927,847.079013
1924,1941,1948,1919,98,747.056070,5,1928,564.051561
...,...,...,...,...,...,...,...,...
1992,2009,2016,1987,30,843.394858,21,2012,1336.713717
1992,2009,2016,1987,30,926.765634,22,2013,1340.069290
1992,2009,2016,1987,30,885.184222,23,2014,696.864247


In [28]:
sims['CumEnergy']=sims['Energy'].groupby('StartYear').cumsum()

In [29]:
sims['DiscountedEnergy']=sims['Energy']*(1/(discount_rate**(sims['YearNo']-1)))

In [30]:
sims['CumDiscountedEnergy']=sims['DiscountedEnergy'].groupby('StartYear').cumsum()

In [31]:
sims

,DebtEnd,EquityEnd,SimName,EquivalentPeriod,Energy,YearNo,WaterYear,Flow_mean,CumEnergy,DiscountedEnergy,CumDiscountedEnergy
StartYear,,,,,,,,,,,
1924,1941,1948,1919,98,827.686134,1,1924,991.317021,827.686134,827.686134,827.686134
1924,1941,1948,1919,98,767.292232,2,1925,1111.041962,1594.978366,667.210637,1494.896771
1924,1941,1948,1919,98,857.958191,3,1926,990.272631,2452.936557,648.739653,2143.636424
1924,1941,1948,1919,98,826.892271,4,1927,847.079013,3279.828828,543.695090,2687.331514
1924,1941,1948,1919,98,747.056070,5,1928,564.051561,4026.884898,427.131733,3114.463247
...,...,...,...,...,...,...,...,...,...,...,...
1992,2009,2016,1987,30,843.394858,21,2012,1336.713717,16673.648519,51.531661,5340.122294
1992,2009,2016,1987,30,926.765634,22,2013,1340.069290,17600.414153,49.239686,5389.361980
1992,2009,2016,1987,30,885.184222,23,2014,696.864247,18485.598376,40.896032,5430.258012


In [36]:
sims_summary=(sims.loc[sims['YearNo']==debt_period]).drop(['Energy','YearNo','DiscountedEnergy','DebtEnd','EquityEnd'],axis=1).rename(columns={'CumEnergy':'DebtEnergy','CumDiscountedEnergy':'DebtDiscountedEnergy'})

sims_summary=sims_summary.join((sims.loc[sims['YearNo']==equity_period])
            .drop(['Energy','YearNo','DiscountedEnergy','SimName','EquivalentPeriod','WaterYear','Flow_mean'],axis=1)
            .rename(columns={'CumEnergy':'EquityEnergy','CumDiscountedEnergy':'EquityDiscountedEnergy'}))

sims_summary.head(3)

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy
StartYear,,,,,,,,,,
1924,1919,98,1941,649.122289,14843.198628,5776.958973,1941,1948,20390.234536,6087.704010
1925,1920,97,1942,668.255065,14824.139120,5766.806111,1942,1949,20317.085109,6075.379756
1926,1921,96,1943,926.766862,14902.268031,5828.002470,1943,1950,20395.091677,6133.830559


In [37]:
sims_summary['DebtAvEnergy']=sims.loc[sims['YearNo']<=debt_period]['Energy'].groupby('StartYear').mean()

In [38]:
sims_summary['EquityAvEnergy']=sims.loc[sims['YearNo']<=equity_period]['Energy'].groupby('StartYear').mean()

In [39]:
sims_summary['DebtAvFlow']=sims.loc[sims['YearNo']<=debt_period]['Flow_mean'].groupby('StartYear').mean()
sims_summary['EquityAvFlow']=sims.loc[sims['YearNo']<=equity_period]['Flow_mean'].groupby('StartYear').mean()

In [40]:
sims_summary

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy,DebtAvEnergy,EquityAvEnergy,DebtAvFlow,EquityAvFlow
StartYear,,,,,,,,,,,,,,
1924,1919,98,1941,649.122289,14843.198628,5776.958973,1941,1948,20390.234536,6087.704010,824.622146,815.609381,924.311172,932.175647
1925,1920,97,1942,668.255065,14824.139120,5766.806111,1942,1949,20317.085109,6075.379756,823.563284,812.683404,906.363285,947.439554
1926,1921,96,1943,926.766862,14902.268031,5828.002470,1943,1950,20395.091677,6133.830559,827.903779,815.803667,896.125780,951.929505
1927,1922,95,1944,996.300978,14840.219516,5789.511533,1944,1951,20343.518819,6095.423716,824.456640,813.740753,896.460688,975.999524
1928,1923,94,1945,679.872416,14884.444220,5787.961469,1945,1952,20331.138019,6087.265537,826.913568,813.245521,887.171433,1001.530558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,1983,34,2005,959.490284,13695.418851,5256.850621,2005,2012,19732.680434,5589.486747,760.856603,789.307217,825.928793,1001.604370
1989,1984,33,2006,1457.179503,13689.713803,5154.656709,2006,2013,19817.318400,5491.838833,760.539656,792.692736,833.161921,1002.127814
1990,1985,32,2007,1299.672497,13833.298848,5150.275743,2007,2014,19954.284281,5486.086841,768.516603,798.171371,875.687631,1008.633995


In [41]:
fdc=pd.read_csv(output_data + "ngonye_flow_annual_exceedance.csv")
sims_summary['DebtEqExceed']=pd.merge_asof(sims_summary.reset_index().sort_values('DebtAvFlow'),fdc.sort_values('Mean'),left_on='DebtAvFlow',right_on='Mean').set_index('StartYear')['Exceedance']
sims_summary['EquityEqExceed']=pd.merge_asof(sims_summary.reset_index().sort_values('EquityAvFlow'),fdc.sort_values('Mean'),left_on='EquityAvFlow',right_on='Mean').set_index('StartYear')['Exceedance']
sims_summary

,SimName,EquivalentPeriod,WaterYear,Flow_mean,DebtEnergy,DebtDiscountedEnergy,DebtEnd,EquityEnd,EquityEnergy,EquityDiscountedEnergy,DebtAvEnergy,EquityAvEnergy,DebtAvFlow,EquityAvFlow,DebtEqExceed,EquityEqExceed
StartYear,,,,,,,,,,,,,,,,
1924,1919,98,1941,649.122289,14843.198628,5776.958973,1941,1948,20390.234536,6087.704010,824.622146,815.609381,924.311172,932.175647,0.653,0.649
1925,1920,97,1942,668.255065,14824.139120,5766.806111,1942,1949,20317.085109,6075.379756,823.563284,812.683404,906.363285,947.439554,0.657,0.636
1926,1921,96,1943,926.766862,14902.268031,5828.002470,1943,1950,20395.091677,6133.830559,827.903779,815.803667,896.125780,951.929505,0.660,0.630
1927,1922,95,1944,996.300978,14840.219516,5789.511533,1944,1951,20343.518819,6095.423716,824.456640,813.740753,896.460688,975.999524,0.659,0.587
1928,1923,94,1945,679.872416,14884.444220,5787.961469,1945,1952,20331.138019,6087.265537,826.913568,813.245521,887.171433,1001.530558,0.662,0.551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1988,1983,34,2005,959.490284,13695.418851,5256.850621,2005,2012,19732.680434,5589.486747,760.856603,789.307217,825.928793,1001.604370,0.708,0.551
1989,1984,33,2006,1457.179503,13689.713803,5154.656709,2006,2013,19817.318400,5491.838833,760.539656,792.692736,833.161921,1002.127814,0.704,0.551
1990,1985,32,2007,1299.672497,13833.298848,5150.275743,2007,2014,19954.284281,5486.086841,768.516603,798.171371,875.687631,1008.633995,0.669,0.547


In [43]:
sims_summary.to_csv(output_data + out_prefix + "_pe_sims_summary.csv")

In [44]:
sims.to_csv(output_data + out_prefix + "_pe_sims.csv")